<h2> 4.Featurizing text data with tfidf weighted word-vectors </h2>

In [8]:
#importing useful libraries
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import plotly.express as px
import plotly.graph_objects as go
sns.color_palette('bright')
sns.set(style='darkgrid',rc = {'figure.figsize':(15,8)})
from plotly.offline import iplot
from sklearn.pipeline import make_pipeline 
%matplotlib inline
import os
print("Ready,set,go....")

Ready,set,go....


In [9]:
#geeting the current path
print(os.getcwd())

/content


In [10]:
#reading data sets
df = pd.read_csv('train.csv')

In [11]:
#checking head
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [12]:
#checking info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404290 entries, 0 to 404289
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            404290 non-null  int64 
 1   qid1          404290 non-null  int64 
 2   qid2          404290 non-null  int64 
 3   question1     404289 non-null  object
 4   question2     404288 non-null  object
 5   is_duplicate  404290 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 18.5+ MB


In [13]:
# encoding question1 to get unicode
df['question1'] = df['question1'].apply(lambda var: str(var))


In [14]:
# encoding question2 to get unicode
df['question2'] = df['question2'].apply(lambda var: str(var))

In [15]:
#checking head after encoding
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [16]:
#checking info after encoding
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404290 entries, 0 to 404289
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            404290 non-null  int64 
 1   qid1          404290 non-null  int64 
 2   qid2          404290 non-null  int64 
 3   question1     404290 non-null  object
 4   question2     404290 non-null  object
 5   is_duplicate  404290 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 18.5+ MB


<h3>4.1 Tfidfvectorizer and Countvectorizer

<h4>References</h4>

- https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

- https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

In [17]:
#importing useful libraries
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [18]:
# merging the texts
questions = list(df['question1']) + list(df['question2'])

In [19]:
#creating instance 
tfidf = TfidfVectorizer(lowercase=False )

In [20]:
#fitting and transform for merging questions
tfidf.fit_transform(questions)

<808580x109679 sparse matrix of type '<class 'numpy.float64'>'
	with 8146555 stored elements in Compressed Sparse Row format>

In [21]:
#checking the question
questions[0]

'What is the step by step guide to invest in share market in india?'

In [22]:
#creating dict key value:word and value:tf-idf scores
word2tfidf = dict(zip(tfidf.get_feature_names(), tfidf.idf_))

<h3>4.2 spacy</h3>

<h4>References</h4>

- https://spacy.io/usage/linguistic-features#vectors-similarity

In [23]:
#importing spacy
import spacy

In [24]:
# After finding the corresponding TF-IDF scores,converting each question to a weighted average of word2vec vectors by these scores.
# en_vectors_web_lg, which includes over 1 million unique vectors.
nlp = spacy.load('en_core_web_sm')
#empty list 
vecs1 = []


<h4>Refernce</h4>

-https://github.com/tqdm/tqdm

In [28]:
#importing tqdm
from tqdm import tqdm
#tqdm is used for printing the progress bar

In [29]:
#method for vecs1
for q1 in tqdm(list(df['question1'])):
    doc1 = nlp(q1) 
    mean_vec1 = np.zeros([len(doc1), len(doc1[0].vector)])
    for word1 in doc1:
        # word2vec
        vec1 = word1.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word1)]
        except:
            idf = 0
        # compute final vec
        mean_vec1 += vec1 * idf
    #mean_vec1 = mean_vec1.mean(axis=0)
    vecs1.append(mean_vec1)
    
df['q1_feats_m'] = list(vecs1)


100%|██████████| 404290/404290 [1:09:32<00:00, 96.90it/s]


In [31]:
#same for question2
vecs2 = []
for q2 in tqdm(list(df['question2'])):
    doc2 = nlp(q2) 
    mean_vec2 = np.zeros([len(doc1), len(doc2[0].vector)])
    for word2 in doc2:
        # word2vec
        vec2 = word2.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word2)]
        except:
            #print word
            idf = 0
        # compute final vec
        mean_vec2 += vec2 * idf
    mean_vec2 = mean_vec2.mean(axis=0)
    vecs2.append(mean_vec2)
df['q2_feats_m'] = list(vecs2)

100%|██████████| 404290/404290 [1:09:15<00:00, 97.28it/s]


In [32]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_feats_m,q2_feats_m
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,"[[-6.179506778717041, 37.45073118805885, -67.9...","[-14.616980731487274, 59.75548753142357, -53.2..."
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,"[[9.236667931079865, -80.37141644954681, -45.7...","[-3.5657422859221697, -16.844570636749268, -13..."
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,"[[97.54682850837708, 22.972195133566856, -39.5...","[156.8336295336485, 59.99189615249634, -8.4143..."
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,"[[57.58699941635132, -22.017087638378143, -4.5...","[41.47243919968605, 56.71731689572334, 31.5306..."
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,"[[83.1857842206955, -40.50698482990265, -83.40...","[-14.446974992752075, -4.33825546503067, -70.1..."


In [34]:
#reading nlpfeatures dataframe
df_nlp = pd.read_csv("Nlp_Featuresa.csv",encoding='latin-1')

In [36]:
#reading for_A_fe.csv
df_ppro = pd.read_csv("for_A_f_e.csv",encoding='latin-1')

In [37]:
df1 = df_nlp.drop(['qid1','qid2','question1','question2'],axis=1)


In [38]:
df2 = df_ppro.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)

In [39]:
df3 = df.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)

In [40]:
df3_q1 = pd.DataFrame(df3.q1_feats_m.values.tolist(), index= df3.index)

/usr/local/lib/python3.7/dist-packages/pandas/core/internals/construction.py:305: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray



In [41]:
df3_q2 = pd.DataFrame(df3.q2_feats_m.values.tolist(), index= df3.index)

In [42]:
#checking head
df_nlp.head()

,id,qid1,qid2,question1,question2,is_duplicate,common_word,Total_words,share_words,Q1len,Q2len,Q1_nwords,Q2_nwords,frequency_qid1,frequency_qid2,frequency_q1+q2,frequency_q1-q2,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,token_set_ratio,token_sort_ratio,fuzz_ratio,fuzz_partial_ratio,longest_substr_ratio
0,0,1,2,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,0,10.0,23.0,0.434783,66,57,14,12,1,1,2,0,0.999980,0.833319,0.999983,0.999983,0.916659,0.785709,0.0,1.0,2.0,13.0,100,93,93,100,0.982759
1,1,3,4,what is the story of kohinoor koh i noor dia...,what would happen if the indian government sto...,0,4.0,20.0,0.200000,51,88,8,13,4,1,5,3,0.799984,0.399996,0.749981,0.599988,0.699993,0.466664,0.0,1.0,5.0,12.5,86,63,66,75,0.596154
2,2,5,6,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...,0,4.0,24.0,0.166667,73,59,14,10,1,1,2,0,0.399992,0.333328,0.399992,0.249997,0.399996,0.285712,0.0,1.0,4.0,12.0,63,63,43,47,0.166667
3,3,7,8,why am i mentally very lonely how can i solve...,find the remainder when math 23 24 math i...,0,0.0,19.0,0.000000,50,65,11,9,1,1,2,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,2.0,12.0,28,24,9,14,0.039216
4,4,9,10,which one dissolve in water quikly sugar salt...,which fish would survive in salt water,0,2.0,20.0,0.100000,76,39,13,7,3,1,4,2,0.399992,0.199998,0.999950,0.666644,0.571420,0.307690,0.0,1.0,6.0,10.0,67,47,35,56,0.175000


In [43]:
#checking head
df_ppro.head()

,id,qid1,qid2,question1,question2,is_duplicate,common_word,Total_words,share_words,Q1len,Q2len,Q1_nwords,Q2_nwords,frequency_qid1,frequency_qid2,frequency_q1+q2,frequency_q1-q2
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,10.0,23.0,0.434783,66,57,14,12,1,1,2,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,4.0,20.0,0.200000,51,88,8,13,4,1,5,3
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,4.0,24.0,0.166667,73,59,14,10,1,1,2,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,0.0,19.0,0.000000,50,65,11,9,1,1,2,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,2.0,20.0,0.100000,76,39,13,7,3,1,4,2


In [44]:
#checking head
df1.head()

,id,is_duplicate,common_word,Total_words,share_words,Q1len,Q2len,Q1_nwords,Q2_nwords,frequency_qid1,frequency_qid2,frequency_q1+q2,frequency_q1-q2,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,token_set_ratio,token_sort_ratio,fuzz_ratio,fuzz_partial_ratio,longest_substr_ratio
0,0,0,10.0,23.0,0.434783,66,57,14,12,1,1,2,0,0.999980,0.833319,0.999983,0.999983,0.916659,0.785709,0.0,1.0,2.0,13.0,100,93,93,100,0.982759
1,1,0,4.0,20.0,0.200000,51,88,8,13,4,1,5,3,0.799984,0.399996,0.749981,0.599988,0.699993,0.466664,0.0,1.0,5.0,12.5,86,63,66,75,0.596154
2,2,0,4.0,24.0,0.166667,73,59,14,10,1,1,2,0,0.399992,0.333328,0.399992,0.249997,0.399996,0.285712,0.0,1.0,4.0,12.0,63,63,43,47,0.166667
3,3,0,0.0,19.0,0.000000,50,65,11,9,1,1,2,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,2.0,12.0,28,24,9,14,0.039216
4,4,0,2.0,20.0,0.100000,76,39,13,7,3,1,4,2,0.399992,0.199998,0.999950,0.666644,0.571420,0.307690,0.0,1.0,6.0,10.0,67,47,35,56,0.175000


In [45]:
print("Number of features in nlp dataframe :", df1.shape[1])

Number of features in nlp dataframe : 28


In [46]:
#checking head
df2.head()

,id,common_word,Total_words,share_words,Q1len,Q2len,Q1_nwords,Q2_nwords,frequency_qid1,frequency_qid2,frequency_q1+q2,frequency_q1-q2
0,0,10.0,23.0,0.434783,66,57,14,12,1,1,2,0
1,1,4.0,20.0,0.200000,51,88,8,13,4,1,5,3
2,2,4.0,24.0,0.166667,73,59,14,10,1,1,2,0
3,3,0.0,19.0,0.000000,50,65,11,9,1,1,2,0
4,4,2.0,20.0,0.100000,76,39,13,7,3,1,4,2


In [47]:
print("Number of features in preprocessed dataframe :", df2.shape[1])

Number of features in preprocessed dataframe : 12


In [48]:
#checking head
df3.head()

,id,q1_feats_m,q2_feats_m
0,0,"[[-6.179506778717041, 37.45073118805885, -67.9...","[-14.616980731487274, 59.75548753142357, -53.2..."
1,1,"[[9.236667931079865, -80.37141644954681, -45.7...","[-3.5657422859221697, -16.844570636749268, -13..."
2,2,"[[97.54682850837708, 22.972195133566856, -39.5...","[156.8336295336485, 59.99189615249634, -8.4143..."
3,3,"[[57.58699941635132, -22.017087638378143, -4.5...","[41.47243919968605, 56.71731689572334, 31.5306..."
4,4,"[[83.1857842206955, -40.50698482990265, -83.40...","[-14.446974992752075, -4.33825546503067, -70.1..."


In [49]:
#checking head
df3_q1.head()

,0
0,"[[-6.179506778717041, 37.45073118805885, -67.9..."
1,"[[9.236667931079865, -80.37141644954681, -45.7..."
2,"[[97.54682850837708, 22.972195133566856, -39.5..."
3,"[[57.58699941635132, -22.017087638378143, -4.5..."
4,"[[83.1857842206955, -40.50698482990265, -83.40..."


In [50]:
print("Number of features in question1 w2v  dataframe :", df3_q1.shape[1])

Number of features in question1 w2v  dataframe : 1


In [51]:
#checking head
df3_q2.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95
0,-14.616981,59.755488,-53.263745,19.514497,113.916473,101.657056,8.561499,66.232769,32.888127,210.812733,-14.521656,5.732026,40.648458,-95.787979,-1.157125,-66.563451,-31.051373,-17.968147,-30.063745,32.889709,65.430540,54.742656,1.756501,27.786168,-46.373962,-25.822312,51.231668,-20.655695,98.152685,-96.048390,126.935466,-28.066691,82.523253,-91.751655,-24.696691,-67.690101,171.840663,-62.797236,-22.777055,-60.322768,...,120.843786,26.592192,-50.285062,114.594113,100.273874,-72.797047,85.467161,42.393031,-18.239859,-103.664753,68.317974,-46.449020,-60.184247,47.269530,-113.987847,10.320980,0.135103,-29.869163,-78.649177,77.262293,131.287160,-123.969027,20.822789,20.718112,38.093118,-85.086696,-61.891319,3.915410,-58.229361,-49.735812,-72.266625,-37.072086,-31.142730,94.064854,-45.053242,-34.155221,-76.548099,99.282776,50.791731,-17.566246
1,-3.565742,-16.844571,-130.911785,0.320254,79.350278,23.562028,79.124551,84.119839,128.684135,279.539877,51.329233,-47.977027,124.322700,-106.766716,-35.756386,-119.277429,21.692362,79.320570,-30.874480,-86.451965,-75.311001,-10.504030,-54.459205,-82.283126,-122.537507,-23.897150,38.062149,-120.668866,96.555255,-172.800629,71.174992,52.399280,-3.790546,-88.446559,-1.359644,-169.229485,242.162139,-106.348765,-118.454241,-81.356276,...,190.348300,114.510458,-79.321841,-65.284250,-69.147191,-111.427223,18.650104,15.329342,43.115542,31.039919,49.603525,-26.932295,-54.213027,98.597791,-72.848590,87.272756,2.563240,-12.326506,-50.862417,19.058227,180.085690,-79.230389,-49.612403,-2.875509,12.640813,-205.569995,-46.432224,86.927369,2.640828,-2.350201,6.193171,-65.084229,-15.654534,-3.475828,26.999802,170.172613,-57.038953,194.269546,128.207803,55.490061
2,156.833630,59.991896,-8.414311,29.251426,133.680218,112.457566,89.849781,21.613022,24.331766,171.114490,-104.683288,-27.202060,-45.827002,-96.844532,-12.986917,43.818585,20.793440,15.393396,0.015373,-28.024086,-12.633978,11.869804,-52.478160,-3.826736,8.331102,21.353006,-36.579113,-14.109747,43.868416,-107.133660,125.654210,-0.800086,39.060021,-50.904144,37.683092,-78.781466,122.383833,-67.524980,-107.706957,-19.469239,...,13.483461,-63.411094,40.414527,37.971310,110.463600,-76.722634,-13.193802,38.792827,-49.570232,-142.726638,12.245693,-54.343166,-60.057482,102.482628,3.656340,-21.436032,37.960952,-25.729971,23.332562,9.404027,153.432356,-76.191487,-20.813622,-27.269100,11.735237,-20.692981,-47.580536,42.119569,49.291951,-59.756461,-26.185226,-19.283218,75.602438,24.144027,-91.874398,-178.454113,-91.471482,19.922719,21.266690,49.574858
3,41.472439,56.717317,31.530616,-5.520164,33.454800,79.596179,15.508996,40.042066,21.094017,101.998116,-70.692718,-62.091010,-21.603829,-52.963033,-6.126429,23.129693,-39.828342,-7.557531,10.800786,12.595844,32.663719,-36.700902,-20.552382,-7.180278,-32.160210,28.720635,-42.108100,-63.517539,36.148466,3.026077,-8.988254,64.167554,33.157036,-21.770225,-14.935299,0.127441,61.023365,-50.604968,-81.644456,-43.092263,...,42.726956,10.111637,15.897465,30.342897,42.910006,-101.291523,30.792590,-33.975097,-68.384557,-21.621304,20.336738,18.368145,-2.009289,46.823555,14.814505,84.138001,59.194556,-36.240492,30.679147,-0.383225,48.515156,-53.941062,-25.314894,-6.555996,-48.458986,-34.730500,24.471891,81.124125,-26.392446,-64.587732,-17.779019,30.152297,49.300137,27.783795,25.937188,-32.107076,-3.817634,-14.231000,4.772115,7.711628
4,-14.446975,-4.338255,-70.196208,-48.636382,18.356858,-50.807069,24.311196,60.043674,32.421993,57.148702,-39.954876,-1.186079,103.167159,-60.704754,8.761264,27.956269,11.443861,-44.191169,-44.108188,49.248302,18.939539,29.493630,-19.378594,3.728311,-41.595879,6.727060,46.843683,-32.771427,52.378441,-32.777207,73.263024,12.407717,35.007572,-44.449785,-1.699862,-68.

In [52]:
print("Number of features in question2 w2v  dataframe :", df3_q2.shape[1])

Number of features in question2 w2v  dataframe : 96


In [53]:
print("Number of features in final dataframe  :", df1.shape[1]+df2.shape[1]+df3_q1.shape[1]+df3_q2.shape[1])

Number of features in final dataframe  : 137


In [54]:
df3_q1['id']=df1['id']
df3_q2['id']=df1['id']

In [55]:
df1  = df1.merge(df2, on='id',how='left')
df2  = df3_q1.merge(df3_q2, on='id',how='left')

In [56]:
result  = df1.merge(df2, on='id',how='left')
#saving for fruther use
result.to_csv('final_features.csv')